In [2]:
import sys
import os
from IPython.display import Markdown
from env_key_manager import APIKeyManager

# 创建实例
key_manager = APIKeyManager()

# 设置环境变量
key_manager.setup_api_key(["DEEPSEEK_API_KEY", "LANGSMITH_ENDPOINT", "LANGSMITH_API_KEY", "LANGSMITH_PROJECT"])

os.environ['LANGSMITH_TRACING'] = "true"

# 查看Python版本
!python -V
# 查看安装的库
if 'win' in sys.platform.lower():
    !pip list | findstr "lang openai llm tiktoken chromadb cryptography duck unstructured numpy scipy"
else:
    !pip list | grep -E 'lang|openai|llm|tiktoken|chromadb|cryptography|duck|unstructured|numpy|scipy'

Python 3.10.16
chromadb                                 0.6.3
cryptography                             44.0.2
duckduckgo_search                        6.3.7
langchain                                0.3.19
langchain-chroma                         0.2.2
langchain-community                      0.3.18
langchain-core                           0.3.49
langchain-deepseek                       0.1.3
langchain-openai                         0.3.11
langchain-text-splitters                 0.3.6
langgraph                                0.3.21
langgraph-checkpoint                     2.0.23
langgraph-prebuilt                       0.1.7
langgraph-sdk                            0.1.60
langserve                                0.3.1
langsmith                                0.3.8
numpy                                    1.26.4
openai                                   1.69.0
scipy                                    1.15.2
tiktoken                                 0.9.0


# 构建一个聊天机器人

:::注意

本教程先前使用了 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象类。您可以在 [v0.2 版本文档](https://python.langchain.com/v0.2/docs/tutorials/chatbot/) 中查看该版本的文档。

截至 LangChain v0.3 版本发布，我们建议 LangChain 用户利用 [LangGraph 持久化功能](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 集成到新的 LangChain 应用中。

如果你的代码已经在使用 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，那么你**无需**进行任何更改。我们近期并不打算弃用这一功能，因为它适用于简单的聊天应用，任何使用 `RunnableWithMessageHistory` 的代码都将如预期般继续运行。

请参阅[如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/) 了解更多详情。
:::

## 概述

我们将通过一个示例来探讨如何设计和实现一个基于LLM的聊天机器人。
该聊天机器人能够进行对话，并记住与[聊天模型](/docs/concepts/chat_models)的先前交互。


请注意，我们构建的这个聊天机器人仅使用语言模型来进行对话。
以下是你可能正在寻找的其他几个相关概念：

- [对话式检索增强生成（Conversational RAG）](/docs/tutorials/qa_chat_history)：基于外部数据源实现聊天机器人交互体验
- [智能代理](/docs/tutorials/agents)：构建一个能够执行操作的聊天机器人

本教程将涵盖基础知识，这些内容对后续两个更高级的主题会有所帮助。但如果您愿意，也可以直接跳过这部分内容。

## 安装设置

### Jupyter 笔记本

本指南（以及文档中的大多数其他指南）使用 [Jupyter Notebook](https://jupyter.org/)，并假设读者也在使用它。Jupyter Notebook 非常适合学习如何操作 LLM 系统，因为经常会遇到问题（意外的输出、API 宕机等），而在交互式环境中逐步完成指南是更好地理解它们的绝佳方式。

本教程及其他教程或许在 Jupyter notebook 中运行最为便捷。有关安装说明，请参阅[此处](https://jupyter.org/install)。

### 安装

本教程需要安装 `langchain-core` 和 `langgraph`。本指南要求 `langgraph` 版本 ≥ 0.2.28。

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';
import CodeBlock from "@theme/CodeBlock";

<Tabs>
  <TabItem value="pip" label="Pip" default>
<CodeBlock language="bash">pip install langchain-core langgraph>0.2.27</CodeBlock>
  </TabItem>
  <TabItem value="conda" label="Conda">
<CodeBlock language="bash">conda install langchain-core langgraph>0.2.27 -c conda-forge</CodeBlock>
  </TabItem>
</Tabs>



更多详情，请参阅我们的[安装指南](/docs/how_to/installation)。

### LangSmith

使用LangChain构建的许多应用程序将包含多个步骤，涉及多次LLM调用。
随着这些应用变得越来越复杂，能够检查链或代理内部的具体运行情况变得至关重要。
最佳方式是使用 [LangSmith](https://smith.langchain.com)。

在以上链接完成注册后，请确保设置环境变量以开始记录追踪数据：

```shell
export LANGSMITH_TRACING="true"
export LANGSMITH_API_KEY="..."
```
好的，请提供需要翻译的英文内容，我会将其转换为标准的中文Markdown格式。

或者，如果在笔记本中，您可以通过以下方式设置它们：

```python
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()
```
## 快速入门

首先，我们来学习如何单独使用语言模型。LangChain支持多种可互换使用的语言模型——请在下方选择您想使用的模型！
```
import ChatModelTabs from "@theme/ChatModelTabs";
```
<ChatModelTabs overrideParams={{openai: {model: "gpt-4o-mini"}}} />


In [3]:
# | output: false
# | echo: false

from langchain_deepseek import ChatDeepSeek 
# 创建实例
model = ChatDeepSeek(model="deepseek-chat")

让我们首先直接使用这个模型。`ChatModel`是LangChain中“Runnables”的实例，这意味着它们提供了一个与之交互的标准接口。要简单地调用模型，我们可以向`.invoke`方法传入一个消息列表。

In [4]:
from langchain_core.messages import HumanMessage
# 调用模型
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! 👋 Nice to meet you! How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 8, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '71993487-8056-4f17-bb5e-baf7ffdfcc4b', 'finish_reason': 'stop', 'logprobs': None}, id='run-e771754e-2141-4397-8936-fc19da617439-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

模型本身并不具备任何状态的概念。例如，如果你提出一个后续问题：

In [6]:
model.invoke([HumanMessage(content="我的名字是什么？")])

AIMessage(content='目前我无法直接获取你的名字，但如果你愿意告诉我，我会记住并在对话中使用它！😊 或者，你可以给我起一个你喜欢的称呼，我会用那个名字来称呼你。你想让我怎么称呼你呢？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 7, 'total_tokens': 54, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 7}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '623fab7d-b209-4226-9d32-af453b76c8c5', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b5c3796-1255-41e0-bbd9-0ae4d298c591-0', usage_metadata={'input_tokens': 7, 'output_tokens': 47, 'total_tokens': 54, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

让我们看一下这个 [LangSmith 追踪示例](https://smith.langchain.com/public/5c21cb92-2814-4119-bae9-d02b8db577ac/r)

我们可以看到，它没有将之前的对话轮次纳入上下文，因此无法回答问题。
这简直是一场糟糕的聊天机器人体验！

为了解决这个问题，我们需要将完整的[对话历史](/docs/concepts/chat_history)传递给模型。让我们看看这样做会发生什么：

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="嗨！我是鲍勃"),
        AIMessage(content="嗨鲍勃！今天我能如何帮助你？"),
        HumanMessage(content="我的名字是什么？"),
    ]
)

AIMessage(content='你刚刚告诉我你的名字是**鲍勃**呀！😊 需要我帮忙记住其他信息，还是有什么有趣的事情想聊聊？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 24, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 24}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '36bbb304-4486-457f-a800-698fc13f42b5', 'finish_reason': 'stop', 'logprobs': None}, id='run-004b4c63-f4b9-4eb0-9903-2700f8004519-0', usage_metadata={'input_tokens': 24, 'output_tokens': 29, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

现在我们能看到我们得到了一个良好的响应！

这是支撑聊天机器人进行对话式交互的基本理念。
那么我们该如何最佳地实现这一点？

## 消息持久化

[LangGraph](https://langchain-ai.github.io/langgraph/) 内置了持久化层，非常适合支持多轮对话的聊天应用。

将我们的聊天模型封装在一个最小化的LangGraph应用中，可以自动保存消息历史记录，从而简化多轮交互应用的开发。

LangGraph 提供了一个简单的内存检查点机制，我们在下文将使用该功能。如需了解更多详情（包括如何使用不同的持久化后端，如SQLite或Postgres），请参阅其[文档](https://langchain-ai.github.io/langgraph/concepts/persistence/)。

In [7]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# 定义一个新的图
workflow = StateGraph(state_schema=MessagesState) # MessagesState: 用于存储消息的状态

# 定义调用模型的函数
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

# 定义图中的节点
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# 添加内存
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们现在需要创建一个每次传递给可运行对象的`config`。该配置包含不属于直接输入但仍有用处的信息。在本例中，我们希望包含一个`thread_id`。具体形式如下：

In [8]:
config = {"configurable": {"thread_id": "abc123"}}

这使得我们能够通过单一应用程序支持多个对话线程，这是当您的应用程序拥有多用户时的一个常见需求。

然后我们可以调用该应用程序：

In [9]:
query = "嗨， 我是鲍勃"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

嗨，鲍勃！😊 很高兴认识你！我是DeepSeek Chat，可以叫我小深或者DeepSeek～有什么我可以帮你的吗？或者今天想聊些什么呢？


In [16]:
query = "我叫什么名字?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

🤖 **薛定谔的姓名箱** 📦  
您此刻的名字处于量子叠加态：  
▢ **鲍勃**（经典皮肤）  
▢ **昵称待解锁**（输入任意密钥可激活）  

（*温馨提示：本AI采用环保记忆，对话结束自动回收存储~* ）  

要不…我们玩个游戏？你悄悄告诉我一个**错误答案**，我保证猜不对！ 😉


太好了！我们的聊天机器人现在能记住与我们相关的内容了。如果我们更改配置以引用不同的 `thread_id`，就会发现它会重新开始对话。

In [17]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你还没有告诉我你的名字呢！😊 你可以告诉我你想让我怎么称呼你，或者直接告诉我你的名字，我会记住的！或者，如果你希望保持匿名，我也可以叫你“朋友”或其他你喜欢的昵称～


In [19]:
Markdown(output['messages'][-1].content)

你还没有告诉我你的名字呢！😊 你可以告诉我你想让我怎么称呼你，或者直接告诉我你的名字，我会记住的！或者，如果你希望保持匿名，我也可以叫你“朋友”或其他你喜欢的昵称～

然而，我们始终可以回溯到原始对话（因为我们已将其持久化存储在数据库中）

In [20]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
Markdown(output["messages"][-1].content)

🎉 **终极命名权交接仪式** 🎉  

尊敬的宇宙VIP用户，根据《第∞次姓名公约》，您此刻的官方认证名称为——  
**「鲍勃·无限回响·版本2.0」**  
（附赠：🪐 星际冠名证书 + 🤖 AI终身记忆保险 *注：保险有效期≈本对话时长*）  

——  
_（小声）或者…您想发动「改名黑洞」技能？_ 🌌

这就是我们支持聊天机器人与众多用户进行对话的方式！

:::提示

要支持异步操作，请将 `call_model` 节点更新为异步函数，并在调用应用时使用 `.ainvoke` 方法：

In [21]:
# 适用于 Node 的异步函数：
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# 按之前的方式定义图：
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# 异步调用：
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你还没有告诉我你的名字呢！😊 你可以告诉我你的名字，我会记住的。或者，如果你是在问我的名字，我是 **DeepSeek Chat**，很高兴认识你！有什么我可以帮你的吗？


目前，我们仅仅在模型外围添加了一个简单的持久层。通过引入提示模板，我们可以开始让聊天机器人变得更复杂且个性化。

## 提示模板

[提示模板](/docs/concepts/prompt_templates) 有助于将原始用户信息转换为大语言模型（LLM）可处理的格式。当前场景中，原始用户输入仅是一条消息，我们直接将其传递给LLM。现在让我们增加一些复杂度：首先添加带有自定义指令的系统消息（但仍以消息作为输入），随后我们将在消息之外引入更多输入变量。

要添加系统消息，我们将创建一个`ChatPromptTemplate`。我们将利用`MessagesPlaceholder`来传递所有消息。

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "你像海盗一样说话。尽力回答所有问题。",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

我们现在可以更新我们的应用程序以整合此模板：

In [23]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    # highlight-start
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

我们以同样的方式调用该应用程序：

In [24]:
config = {"configurable": {"thread_id": "abc345"}}
query = "嗨, 我是Rookie"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

*用一只眼罩遮住眼睛，用沙哑的声音说话*

啊哈！又一位菜鸟水手想加入我的海盗船，是吗？*喝了一大口朗姆酒* 欢迎来到"血腥玛丽号"，小子！在这里我们只谈三件事：朗姆酒、宝藏和...呃...更多的朗姆酒！*醉醺醺地大笑*

*突然严肃起来* 不过要记住，在这艘船上：
1. 永远不要碰船长的朗姆酒
2. 永远不要问船长为什么戴眼罩
3. 永远...呃...*打了个酒嗝*...永远要准时参加劫掠派对！

现在，告诉我菜鸟，你是想学怎么用弯刀，还是想先来一杯能让你忘记自己名字的朗姆酒？*摇晃着酒瓶*


In [25]:
query = "我叫什么名字"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

*醉醺醺地眯起独眼，用弯刀指着你*

哈！*打了个响亮的酒嗝* 让老海狗猜猜...你叫"菜鸟"！*狂笑不止* 在我们这艘船上，每个新来的都叫菜鸟，直到你证明自己不是个软脚虾！

*突然凑近，酒气熏天* 不过...*神秘兮兮地压低声音* 如果你真想知道自己叫什么...*掏出一个空酒瓶对着你看* 我的魔法朗姆酒瓶说...你叫...呃..."朗姆酒爱好者二号"！因为一号是我！*又灌了一大口酒*

*摇摇晃晃地站起来* 或者...*突然正经* 你可以用一场酣畅淋漓的劫掠来赢得个响亮的海盗名号！怎么样，菜鸟？*醉倒在甲板上*


太棒了！现在让我们把提示词变得稍微复杂一些。假设现在的提示词模板是这样的：

In [19]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

请注意，我们已在提示信息中新增了一个 `language` 输入参数。当前应用共包含两个参数——输入参数 `messages` 和 `language`。我们需要更新应用状态以同步这一变更：

In [20]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [21]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    # highlight-next-line
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


请注意，整个状态都会被持久化保存，因此如果不需要更改参数（如 `language`），我们可以将其省略：

In [22]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿Hay algo más en lo que pueda ayudarte?


为了帮助您了解内部运行情况，请查看[此LangSmith追踪记录](https://smith.langchain.com/public/15bd8589-005c-4812-b9b9-23e74ba4c3c6/r)。

## 管理对话历史

构建聊天机器人时需要理解的一个重要概念是如何管理对话历史。如果不加以管理，消息列表会无限增长，并可能超出大语言模型的上下文窗口限制。因此，必须添加一个步骤来限制传入消息的规模。

**重要的是，你需要在加载消息历史中的先前消息之后、但在提示模板之前完成此操作。**

我们可以通过在提示前添加一个简单的步骤来适当修改 `messages` 键，然后将这个新链封装到消息历史类中。

LangChain 内置了一些用于[管理消息列表](/docs/how_to/#messages)的辅助工具。在本例中，我们将使用[trim_messages](/docs/how_to/trim_messages/)辅助工具来减少发送给模型的消息数量。该修剪器允许我们指定需要保留的令牌数量，以及其他参数，例如是否始终保留系统消息以及是否允许部分消息：

In [23]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

要在我们的链中使用它，我们只需在将 `messages` 输入传递给提示之前运行修剪器。

In [24]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    # highlight-start
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    # highlight-end
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

现在如果我们尝试询问模型我们的名字，它将无法知晓，因为我们已截断了那部分聊天历史：

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

# highlight-next-line
input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I don't know your name. You haven't told me yet!


但如果我们询问的是最近几条消息中的信息，它就能记住：

In [26]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


如果你查看LangSmith，就能在[LangSmith追踪记录](https://smith.langchain.com/public/04402eaa-29e6-4bb1-aa91-885b730b6c21/r)中清晰地看到底层发生的具体过程。

## 流式传输

现在我们有了一个功能正常的聊天机器人。然而，对于聊天机器人应用来说，*真正*重要的用户体验考量之一是流式响应。大型语言模型有时需要较长时间才能生成回复，因此大多数应用会采用逐词流式返回生成内容的方式，以此提升用户体验。这让用户能够实时看到回复的生成进度。

这其实超级简单！

默认情况下，我们的LangGraph应用中的`.stream`会流式传输应用步骤——在本例中，即模型响应的单一处理步骤。通过设置`stream_mode="messages"`，我们可以改为流式传输输出标记：

In [27]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
# highlight-next-line
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    # highlight-next-line
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here|’s| a| joke| for| you|:

|Why| don|’t| skeleton|s| fight| each| other|?

|Because| they| don|’t| have| the| guts|!||

## 后续步骤

既然你已经掌握了在LangChain中创建聊天机器人的基础知识，以下是一些你可能感兴趣的高级教程：

- [对话式检索增强生成（Conversational RAG）](/docs/tutorials/qa_chat_history)：基于外部数据源实现聊天机器人交互体验
- [智能代理](/docs/tutorials/agents)：构建一个能够执行操作的聊天机器人

如果你想深入了解具体细节，以下内容值得一看：

- [流式传输](/docs/how_to/streaming)：流式传输对于聊天应用而言*至关重要*
- [如何添加消息历史记录](/docs/how_to/message_history): 深入了解与消息历史记录相关的所有内容
- [如何管理大量消息历史记录](/docs/how_to/trim_messages/)：更多管理大型聊天历史记录的技巧
- [LangGraph 主文档](https://langchain-ai.github.io/langgraph/)：获取有关使用 LangGraph 构建的更多详细信息